In [1]:
# External libraries used for various tasks in later parts of the code
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import xml.etree.ElementTree as ET
from torch.utils.data import Dataset
from imutils import paths
import torch
import torch.nn as nn
import time
import numpy as np
import torch.optim as optim
from tqdm.notebook import tqdm
import os

# Check if running on Google Colab and set the data paths accordingly
is_running_on_colab = 'COLAB_GPU' in os.environ
if is_running_on_colab:
    from google.colab import drive
    drive.mount('/content/gdrive')
    data_path = '/content/gdrive/MyDrive/license_plate/data/'
    test_path = '/content/gdrive/MyDrive/license_plate/testdata/'
else:
    current_dir = os.getcwd()
    data_path = os.path.join(current_dir, 'dataset', 'license_plate', 'data')
    test_path = os.path.join(current_dir, 'dataset', 'license_plate', 'test_data')

# Function to check if directories exist
def check_directories_exist(*paths):
    for path in paths:
        assert os.path.isdir(path), f"The directory {path} does not exist."

# Check the directories
check_directories_exist(data_path, test_path)

data_link = "https://drive.google.com/open?id=1rdEsCUcIUaYOVRkx5IMTRNA7PcGMmSgc"

Mounted at /content/gdrive


In [2]:
from torch.utils.data import Dataset
import cv2
from imutils import paths
import os
import torch

class LabelFpsDataLoader(Dataset):
    def __init__(self, img_dirs, img_size, is_transform=None):
        self.img_paths = [p for img_dir in img_dirs for p in paths.list_images(img_dir)]
        self.img_size = img_size
        self.is_transform = is_transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, index):
        img_name = self.img_paths[index]
        img = cv2.imread(img_name)
        resized_image = self.transform_image(img)

        file_name = os.path.splitext(os.path.basename(img_name))[0]
        labels = self.extract_labels(file_name, img.shape)

        return resized_image, labels, img_name

    def transform_image(self, img):
        resized_image = cv2.resize(img, self.img_size).astype('float32') / 255.0
        # print(resized_image.dtype)
        # Normalization - Adjust mean and std if necessary
        mean = np.array([0.485, 0.456, 0.406], dtype=resized_image.dtype)
        std = np.array([0.229, 0.224, 0.225],dtype=resized_image.dtype)

        resized_image = (resized_image - mean) / std  # Normalize
        # print(resized_image.dtype)
        return resized_image

    def extract_labels(self, file_name, img_shape):
        parts = file_name.split('-')
        points = [self.parse_point(part) for part in parts[2].split('_')]
        ori_w, ori_h = img_shape[1], img_shape[0]
        scale_w, scale_h = self.img_size[0] / ori_w, self.img_size[1] / ori_h
        scaled_points = [(int(x * scale_w), int(y * scale_h)) for x, y in points]
        flattened_labels = [coord for point in scaled_points for coord in point]
        return torch.tensor(flattened_labels, dtype=torch.float32)

    @staticmethod
    def parse_point(point_str):
        return tuple(int(coord) for coord in point_str.split('&'))


In [3]:
# Initialize your dataset
img_dirs = [data_path]  # Replace with actual paths to your image directories
img_size = (224, 224)  # Replace with your desired image size
dataset = LabelFpsDataLoader(img_dirs=img_dirs, img_size=img_size)  # Use 'img_dirs' as the correct argument name

# Test the length of the dataset
print(f"Length of dataset: {len(dataset)}")
#
# Test getting an item
try:
    # Retrieve the first item
    img, labels, img_name = dataset[0]

    # Check the shapes and types
    print(f"Image shape: {img.shape}")
    print(f"Labels: {labels}")
    print(f"Image name: {img_name}")

    # If the code reaches this point, the item has been retrieved successfully
    print("Item retrieval successful.")
except Exception as e:
    # If there is any error, print it out
    print(f"An error occurred: {e}")
finally:
    print("Got out here")


Length of dataset: 5769
An error occurred: name 'np' is not defined
Got out here


In [4]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

def plot_img_and_boundingbox(image, labels):
    # Assuming image is a PyTorch tensor, we need to first move it to CPU if it's on GPU,
    # then detach it from the current graph, convert to numpy, and finally transpose it
    # from (channels, height, width) to (height, width, channels)
    numpy_array = image.cpu().detach().numpy()

    # Convert from BGR to RGB format if the image was read by OpenCV
    rgb_image = cv2.cvtColor(numpy_array, cv2.COLOR_BGR2RGB)

    # Extract numerical values from tensors and convert to integers
    left_up = (int(labels[0]), int(labels[1]))
    right_down = (int(labels[2]), int(labels[3]))

    # Draw rectangle on the image
    new_image = cv2.rectangle(rgb_image, left_up, right_down, (255, 0, 0), 2)

    # Display the image
    plt.imshow(new_image)
    plt.axis('off')  # Hide the axis
    plt.show()

# Example usage:
# img, labels, _ = dataset[0]  # Assuming this is how you get an image and labels from your dataset
# plot_img_and_boundingbox(img, labels)


In [5]:
import torch
import torch.nn as nn


# Function to create the modified classifier
def create_custom_classifier(input_features, hidden_units, output_features):
    classifier = nn.Sequential(
        nn.Linear(input_features, hidden_units),
        nn.ReLU(),
        nn.Dropout(0.0),
        #nn.BatchNorm1d(hidden_units),
        nn.Linear(hidden_units, output_features),
    )
    return classifier

# Load a pre-trained MobileNetV3 model
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v3_large', pretrained=True)

# Replace the classifier with a new one - adjust the input_features to match the last layer of MobileNetV3
model.classifier = create_custom_classifier(input_features=960, hidden_units=100, output_features=4)
#Sets model to training mode,e this is important for layers that have different functionalities depending on training or eval. for example BatchNorm
model.train()


Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth
100%|██████████| 21.1M/21.1M [00:00<00:00, 82.8MB/s]


MobileNetV3(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): Identity()
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=

In [6]:
# Number of layers you want to fine-tune
layers_of_classifier = len(model.classifier)

# Get all the parameters from the model as a list
parameters = list(model.parameters())

# The total number of layers is the length of the parameters list
number_of_layers = len(parameters)
print(f"Total number of layers is {number_of_layers}")

# Calculate the number of pretrained layers (all except the last 'layers_of_classifier')
pretrained_layers = number_of_layers - layers_of_classifier
print(f"Number of pretrained base layers is {pretrained_layers}")

# Freeze the parameters of the pretrained base layers
for param in parameters[:-layers_of_classifier]:
    param.requires_grad = False

# Unfreeze the parameters of the last 'layers_of_classifier' layers
for param in parameters[-layers_of_classifier:]:
    param.requires_grad = True

Total number of layers is 174
Number of pretrained base layers is 170


In [7]:
def check_requires_grad():
  for name, param in model.named_parameters(): # Just to check
    if name.startswith('classifier'):
        print(f'Layer {name} - requires_grad: {param.requires_grad}')
    if name.startswith('features'):
        print(f'Layer {name} - requires_grad: {param.requires_grad}')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# check_requires_grad()

In [8]:
# from datetime import datetime
# def bb_intersection_over_union(boxA, boxB): # https://pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/
# 	# determine the (x, y)-coordinates of the intersection rectangle
# 	xA = max(boxA[0], boxB[0])
# 	yA = max(boxA[1], boxB[1])
# 	xB = min(boxA[2], boxB[2])
# 	yB = min(boxA[3], boxB[3])
# 	# compute the area of intersection rectangle
# 	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
# 	# compute the area of both the prediction and ground-truth
# 	# rectangles
# 	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
# 	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
# 	# compute the intersection over union by taking the intersection
# 	# area and dividing it by the sum of prediction + ground-truth
# 	# areas - the interesection area
# 	iou = interArea / float(boxAArea + boxBArea - interArea)
# 	# return the intersection over union value
# 	return iou

# def calculate_true_positives_in_batch(y_pred, y_batch):
#   threshold = 0.5
#   true_positive_amount = 0
#   for i in range(y_pred.shape[0]):
#     y_pred_numpy = y_pred[i].cpu().detach().numpy()
#     y_batch_numpy = y_batch[i].cpu().detach().numpy()
#     iou = bb_intersection_over_union(y_pred_numpy, y_batch_numpy)
#     if iou > threshold:
#       true_positive_amount +=1
#   return true_positive_amount

# #wrapper of tqdm to enable toggling it off easily
# def toggle_tqdm(iterable, use_tqdm=True):
#   if use_tqdm:
#     return tqdm(iterable)
#   else:
#     return iterable

# def calculate_loss_and_accuracy(loader):
#   total_loss = 0
#   total_true_positives = 0
#   total_samples = 0

#   for X_batch_, y_batch, img_name in loader:
#       X_batch = X_batch_.permute(0,3,1,2).to(device)
#       y_batch = y_batch.to(device)
#       y_pred = model(X_batch)

#       total_true_positives += calculate_true_positives_in_batch(y_pred, y_batch)

#       loss = loss_fn(y_pred, y_batch)
#       total_loss += loss.item() * len(y_batch)  # Accumulate scaled loss
#       total_samples += len(y_batch)  # Accumulate number of samples
#   sample_loss = total_loss / total_samples  # Calculate average loss
#   accuracy = total_true_positives / total_samples
#   return sample_loss, accuracy

# def save_model(model):
#   # Get the current time
#   current_time = datetime.now()

#   # Format the time in a 'YearMonthDay_HourMinuteSecond' format for the filename
#   timestamp = current_time.strftime('%Y%m%d_%H%M')
#   model_path = f'/content/gdrive/MyDrive/license_plate/models/model_weights_{timestamp}.pth'
#   torch.save(model.state_dict(), model_path)

# def load_model(path):
#   model.load_state_dict(torch.load(path))


In [9]:
def save_model(model):
  # Get the current time
  current_time = datetime.now()

  # Format the time in a 'YearMonthDay_HourMinuteSecond' format for the filename
  timestamp = current_time.strftime('%Y%m%d_%H%M')
  model_path = f'/content/gdrive/MyDrive/license_plate/models/model_weights_{timestamp}.pth'
  torch.save(model.state_dict(), model_path)

def load_model(path):
  model.load_state_dict(torch.load(path))

In [10]:
def bb_intersection_over_union(boxA, boxB):
    # Compute the intersection over union on GPU
    # Assumes boxA and boxB are tensors
    xA = torch.max(boxA[..., 0], boxB[..., 0])
    yA = torch.max(boxA[..., 1], boxB[..., 1])
    xB = torch.min(boxA[..., 2], boxB[..., 2])
    yB = torch.min(boxA[..., 3], boxB[..., 3])

    interArea = torch.clamp(xB - xA, min=0) * torch.clamp(yB - yA, min=0)

    boxAArea = (boxA[..., 2] - boxA[..., 0]) * (boxA[..., 3] - boxA[..., 1])
    boxBArea = (boxB[..., 2] - boxB[..., 0]) * (boxB[..., 3] - boxB[..., 1])

    iou = interArea / (boxAArea + boxBArea - interArea)
    return iou

def calculate_true_positives_in_batch(y_pred, y_batch, threshold=0.5):
    # Calculate IoU for the batch
    iou = bb_intersection_over_union(y_pred, y_batch)

    # Count true positives in the batch
    true_positives = torch.sum(iou > threshold).item()
    return true_positives

def calculate_loss_and_accuracy(loader, model, loss_fn, device):
    model.eval()  # Set model to evaluation mode
    total_loss = 0
    total_true_positives = 0
    total_samples = 0

    with torch.no_grad():  # Disable gradient computation
        for X_batch, y_batch, _ in loader:
            X_batch = X_batch.permute(0,3,1,2).to(device)
            y_batch = y_batch.to(device)
            y_pred = model(X_batch)

            total_true_positives += calculate_true_positives_in_batch(y_pred, y_batch)

            loss = loss_fn(y_pred, y_batch)
            total_loss += loss.item() * X_batch.size(0)  # Accumulate scaled loss
            total_samples += X_batch.size(0)  # Accumulate number of samples

    sample_loss = total_loss / total_samples  # Calculate average loss
    accuracy = total_true_positives / total_samples  # Calculate accuracy
    return sample_loss, accuracy


In [11]:
# load_model('/content/gdrive/MyDrive/license_plate/models/model_weights_20231106_1410.pth')


In [12]:
import torch
from torch.utils.data import DataLoader, Subset
from datetime import datetime
from tqdm import tqdm

# Constants
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA_PATH = [data_path]
TEST_PATH = [test_path]
BATCH_SIZE = 2
VAL_BATCH_SIZE = 2
SUBSET_INDICES = 16
SUBSET_VAL_INDICES = 16
DECODER_EPOCHS = 50
N_EPOCHS = 100
USE_TQDM = True
USE_SUBSET = True
PRINT_EVERY = 5

# Define model, loss function, optimizers, and schedulers
model.to(DEVICE)
loss_fn = nn.MSELoss()
optimizer_decoder = optim.AdamW(model.parameters(), lr=1e-5)
# scheduler_decoder = optim.lr_scheduler.OneCycleLR(optimizer_decoder, 1e-2, total_steps=DECODER_EPOCHS)
scheduler_decoder = optim.lr_scheduler.CyclicLR(optimizer_decoder, base_lr=1e-5, max_lr=1e-3, cycle_momentum=False, step_size_up=DECODER_EPOCHS/6)
optimizer_full = optim.AdamW(model.parameters(), lr=5e-6)
scheduler_full = optim.lr_scheduler.CyclicLR(optimizer_decoder, base_lr=1e-5, max_lr=1e-4, cycle_momentum=False, step_size_up=N_EPOCHS/6)

# Define dataloaders
train_dataset = LabelFpsDataLoader(DATA_PATH, (500, 1000))
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_dataset = LabelFpsDataLoader(TEST_PATH, (500, 1000))
validation_loader = DataLoader(val_dataset, batch_size=VAL_BATCH_SIZE, shuffle=True, num_workers=2)

# Use subsets for faster iterations during development
if USE_SUBSET:
    train_dataset = Subset(train_dataset, indices=range(SUBSET_INDICES))
    val_dataset = Subset(val_dataset, indices=range(SUBSET_VAL_INDICES))
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
    validation_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

# Training functions
def train_epoch(dataloader, optimizer, model, loss_fn, scaler):
    for X_batch, y_batch, _ in dataloader:
        optimizer.zero_grad()
        X_batch = X_batch.permute(0, 3, 1, 2).to(DEVICE)
        y_batch = y_batch.to(DEVICE)
        with torch.cuda.amp.autocast():
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

def train_model(n_epochs, optimizer, scheduler, dataloader, val_loader, model, loss_fn):
    train_loss_history = []
    val_loss_history = []
    train_accuracy_history = []
    val_accuracy_history = []
    scaler = torch.cuda.amp.GradScaler()

    for epoch in tqdm(range(n_epochs)):
        model.train()
        # print(f"epoch: {epoch}")
        train_epoch(dataloader, optimizer, model, loss_fn, scaler)
        scheduler.step()

        if epoch % PRINT_EVERY == 0:
            model.eval()
            with torch.no_grad():
                # In-sample
                in_sample_loss, in_sample_accuracy = calculate_loss_and_accuracy(dataloader, model, loss_fn, device)

                train_loss_history.append(in_sample_loss)
                train_accuracy_history.append(in_sample_accuracy)

                # Out-sample
                out_sample_loss, out_sample_accuracy = calculate_loss_and_accuracy(val_loader, model, loss_fn, device)
                val_loss_history.append(out_sample_loss)
                val_accuracy_history.append(out_sample_accuracy)

                if USE_TQDM:
                    tqdm.write(f'Epoch {epoch + 1}, In-sample Loss: {in_sample_loss:.4f}, In-sample Accuracy: {in_sample_accuracy:.4f}')
                    tqdm.write(f'Epoch {epoch + 1}, Out-sample Loss: {out_sample_loss:.4f}, Out-sample Accuracy: {out_sample_accuracy:.4f}')

    return train_loss_history, val_loss_history, train_accuracy_history, val_accuracy_history





In [ ]:
# Training and validation functions are assumed to be defined above this script

# Train the model with the decoder first
print("Starting training with decoder...")
decoder_loss_history, decoder_val_loss_history, decoder_test_accuracy_history, decoder_val_accuracy_history = train_model(
    n_epochs=DECODER_EPOCHS,
    optimizer=optimizer_decoder,
    scheduler=scheduler_decoder,
    dataloader=train_loader,
    val_loader=validation_loader,
    model=model,
    loss_fn=loss_fn
)
save_model(model)
# print("Decoder training completed.")

# Unfreeze model layers for full model training
for param in model.parameters():
    param.requires_grad = True

# Now train the full model
print("Starting full model training...")
full_loss_history, full_val_loss_history, full_test_accuracy_history, full_val_accuracy_history = train_model(
    n_epochs=N_EPOCHS,
    optimizer=optimizer_full,
    scheduler=scheduler_full,
    dataloader=train_loader,  # Assuming train_loader_finetuning is the same as train_loader
    val_loader=validation_loader,
    model=model,
    loss_fn=loss_fn
)
save_model(model)
print("Full model training completed.")


/usr/local/lib/python3.10/dist-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


Starting training with decoder...


  0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
  2%|▏         | 1/50 [00:22<18:23, 22.53s/it]

Epoch 1, In-sample Loss: 121188.9854, In-sample Accuracy: 0.0000
Epoch 1, Out-sample Loss: 124832.2090, Out-sample Accuracy: 0.0000


 12%|█▏        | 6/50 [01:01<07:17,  9.95s/it]

Epoch 6, In-sample Loss: 117071.3301, In-sample Accuracy: 0.0000
Epoch 6, Out-sample Loss: 120483.7646, Out-sample Accuracy: 0.0000


 14%|█▍        | 7/50 [01:06<06:06,  8.53s/it]

In [ ]:
lr_to_test = [(1e-4, 1e-2),(1e-5, 1e-3),(1e-6, 1e-4)]
result_dict =
for lr_base, lr_max in lr_to_test:
  optimizer_decoder = optim.AdamW(model.parameters(), lr=lr_base)
  # scheduler_decoder = optim.lr_scheduler.OneCycleLR(optimizer_decoder, 1e-2, total_steps=DECODER_EPOCHS)
  scheduler_decoder = optim.lr_scheduler.CyclicLR(optimizer_decoder, base_lr=lr_base, max_lr=lr_max, cycle_momentum=False, step_size_up=DECODER_EPOCHS/6)
  optimizer_full = optim.AdamW(model.parameters(), lr=5e-6)
  scheduler_full = optim.lr_scheduler.CyclicLR(optimizer_decoder, base_lr=lr_base, max_lr=lr_max, cycle_momentum=False, step_size_up=N_EPOCHS/6)
  decoder_loss_history, decoder_val_loss_history, decoder_test_accuracy_history, decoder_val_accuracy_history = train_model(
    n_epochs=DECODER_EPOCHS,
    optimizer=optimizer_decoder,
    scheduler=scheduler_decoder,
    dataloader=train_loader,
    val_loader=validation_loader,
    model=model,
    loss_fn=loss_fn
  )




In [ ]:
# initial_lr = 0.00001
# lr_epochs = 100
# optimizer = optim.SGD(model.parameters(), lr=initial_lr)


# lr_hist = []
# train_loss_history = []

# def train_model(n_epochs):
#   i = 0
#   lr = initial_lr
#   for epoch in toggle_tqdm(range(n_epochs), use_tqdm):
#     # print("epoch:", i)
#     for X_batch, y_batch, img_name in toggle_tqdm(trainloader, False):
#       agg_loss = 0
#       X_batch = X_batch.permute(0,3,1,2).to(device) # Needs to have shape [batch_size, channels, height, width]
#       y_batch = y_batch.to(device)
#       y_pred = model(X_batch)
#       loss = loss_fn(y_pred,y_batch)
#       optimizer.zero_grad()
#       loss.backward()
#       optimizer.step()
#       agg_loss += loss.item()
#       train_loss_history.append(agg_loss)

#       lr_hist.append(lr)

#       lr = 1.1*lr
#       for g in optimizer.param_groups:
#         g['lr'] = lr
#       if lr >= 0.0005:
#         break

# train_model(3)




In [ ]:
plt.figure(figsize=(10, 5))
plt.xscale('log')  # Set the y-axis to a logarithmic scale

plt.plot(lr_hist,train_loss_history, label='Train Loss')
plt.xlabel('LR')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()  # Add a legend to differentiate between train and validation loss
plt.show()
print(train_loss_history)
print(lr_hist)

In [ ]:
plt.figure(figsize=(10, 5))
#plt.yscale('log')  # Set the y-axis to a logarithmic scale
plt.plot(test_loss_history, label='Train Loss')
plt.plot(val_loss_history, label='Validation Loss')
plt.title('Loss Curve')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()  # Add a legend to differentiate between train and validation loss
plt.show()
print(val_accuracy_history)

In [ ]:
#print a trainset with predicted label
# train_loader = DataLoader(data_loader, batch_size=1, shuffle=True, num_workers=1)

train_iter = iter(train_loader)

batch = next(train_iter)

X_batch, y_batch, _ = batch

image = batch[0][0]
labels = batch[1][0]

# Get predicted labels
X_batch = X_batch.permute(0,3,1,2).to(device)
pred = model(X_batch)
print(image.shape)
plot_img_and_boundingbox(image, labels)
plot_img_and_boundingbox(image, pred[0])

In [ ]:
# Print a validation image with its predicted labels

validationloader = DataLoader(test_loader, batch_size=1, shuffle=True, num_workers=1)

validationloader_iter = iter(validationloader)

batch = next(validationloader_iter)

X_batch, y_batch, _ = batch
print(X_batch.shape)
print(y_batch.shape)

image = batch[0][0]
labels = batch[1][0]

# Get predicted labels
X_batch = X_batch.permute(0,3,1,2).to(device)
print(X_batch.shape)
pred = model(X_batch)

plot_img_and_boundingbox(image, labels)

In [ ]:
plot_img_and_boundingbox(image,pred[0])
print(pred)
print(calculate_true_positives_in_batch(labels.view(1,4),pred[0].view(1,4))) # Need to change view since we dont have batch